In [1]:
import os
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision import transforms
from captum.robust import PGD, FGSM
import sys
import argparse
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import datetime
from tqdm import tqdm
sys.path.append('../')
from saliency import *
from utils import *
from plots import *
from run import get_model


Bad key "text.kerning_factor" on line 4 in
/home/raza.imam/.conda/envs/xaim/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
/home/raza.imam/.conda/envs/xaim/lib/python3.7/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /home/raza.imam/.conda/envs/xaim/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZNK2at10TensorBase21__dispatch_contiguousEN3c1012MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
DEVICE = 'cuda'
MODEL_PATH = '/home/raza.imam/Documents/HC701B/Project/models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
model = get_model(model_path=MODEL_PATH, device=DEVICE)

transform = transforms.Compose(
            [
                transforms.Grayscale(num_output_channels=3),
                transforms.RandomRotation((90,90)),
                transforms.CenterCrop(200),
                transforms.Resize((224, 224)),
                # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                transforms.ToTensor(),
                ]
            )

# Creating references (means) using Training Set

In [3]:
#global vars
attack_type = 'PDG'
eps = 0.06
cln_folder = f'/home/raza.imam/Documents/XAIM/XAIM/data5/training/{attack_type}_{eps}/Normal/Succ/Clean_x'
atk_folder = f'/home/raza.imam/Documents/XAIM/XAIM/data5/training/{attack_type}_{eps}/Normal/Succ/Succ_x'

In [4]:
# For PGD 0.03
# Let's generate Clean and Adversarial Images' Pertubations, using all Normal Training set.
cln_files = [f for f in os.listdir(cln_folder) if f.endswith(".png")]
images = []
for f in tqdm(cln_files):
    image_path = os.path.join(cln_folder, f)
    image = Image.open(image_path)
    image = transform(image)
    images.append(image)
images_tensor = torch.stack(images)
mean_attns = {}
all_attns = {}
mean_attn_diff = {}
attentions_clean, mean_attns_cln = apply_attn_on_images(model=model, block=-1, images = images_tensor, device=DEVICE)
mean_attns['clean'] = mean_attns_cln
all_attns['clean'] = attentions_clean
image_path = '../plots_succ/clean.png'
plt.imsave(image_path, mean_attns['clean'], cmap='inferno')

100%|██████████| 2677/2677 [00:14<00:00, 178.85it/s]


In [5]:
atk_files = [f for f in os.listdir(atk_folder) if f.endswith(".png")]
images = []
for f in tqdm(atk_files):
    image_path = os.path.join(atk_folder, f)
    image = Image.open(image_path)
    image = transform(image)
    images.append(image)
adv_images_tensor_pdg = torch.stack(images)

attentions_adv_pdg, mean_attns_pdg = apply_attn_on_images(model=model, block=-1, images = adv_images_tensor_pdg, device=DEVICE)
mean_attns_diff_pdg = mean_attns_pdg - mean_attns_cln
mean_attns['PGD'] = mean_attns_pdg
all_attns['PGD'] = attentions_adv_pdg
mean_attn_diff['PGD'] = mean_attns_diff_pdg
image_path = f'../plots_succ/{attack_type}_{eps}.png'
plt.imsave(image_path, mean_attns['PGD'], cmap='inferno')

  0%|          | 0/2677 [00:00<?, ?it/s]

100%|██████████| 2677/2677 [00:14<00:00, 191.15it/s]


# Test attentions generation

In [40]:
test_attack = 'Clean_x'
# eps = 0.06
class_name = 'Normal'

test_folder = f'/home/raza.imam/Documents/XAIM/XAIM/data5/testing/{attack_type}_{eps}/{class_name}/Succ/{test_attack}'
test_files = [f for f in os.listdir(test_folder) if f.endswith(".jpg") or f.endswith(".png")]
images = []
for f in tqdm(test_files):
    image_path = os.path.join(test_folder, f)
    image = Image.open(image_path)
    image = transform(image)
    images.append(image)
test_images_tensor = torch.stack(images)
test_attns = {}
attentions_clean, _ = apply_attn_on_images(model=model, block=-1, images = test_images_tensor, device=DEVICE)
test_attns['clean'] = attentions_clean

100%|██████████| 329/329 [00:01<00:00, 214.48it/s]


In [41]:
len(test_attns['clean'])

329

# Classification

In [44]:
from run import classify_image
gt_labels = []
sum_preds = [] #sum_preds = ssim preds
euc_preds = []
cos_preds = []
test_attack_type = 'clean'

for idx, attn_map in enumerate(test_attns['clean']):
    result = classify_image(
        img_attn_map = attn_map,
        mean_attn_clean = mean_attns['clean'], 
        mean_attn_adv=mean_attns, 
        method='all',
        attacks = ['PGD']
        )
    sum_preds.append(result['sum'])
    euc_preds.append(result['euclidean'])
    cos_preds.append(result['cosine'])
    gt_labels.append(test_attack_type)

results_dict ={
    "GT": gt_labels,
    "sum": sum_preds,
    "euclidean": euc_preds,
    "cosine": cos_preds,
}

# results_df = pd.DataFrame(results_dict)
# os.makedirs("./results", exist_ok=True)
# results_df.to_csv(f"./results/preds_{attack_name}_block_{args.block}_images_{args.num_train_imgs}_eps_{args.eps}_dataset_{args.dataset_class}.csv", index=False)

# 1 = clean
# 0 = adversarial
gt_labels_bin = [1 if label == "clean" else 0 for label in gt_labels]

methods = ["sum", "euclidean", "cosine"]
for method in methods:
    pred_bin = [1 if label == "Clean" else 0 for label in results_dict[method]]
    print(f'--------------- {method} ---------------')
    print(f"Accuracy for {method}: {accuracy_score(gt_labels_bin, pred_bin)}")
    print(f"F1 score for {method}: {f1_score(gt_labels_bin, pred_bin)}")
    print(f'-------------------------------------------------------------------')
    print(f'-------------------------------------------------------------------')

--------------- sum ---------------
Accuracy for sum: 0.5167173252279635
F1 score for sum: 0.6813627254509018
-------------------------------------------------------------------
-------------------------------------------------------------------
--------------- euclidean ---------------
Accuracy for euclidean: 0.5227963525835866
F1 score for euclidean: 0.686626746506986
-------------------------------------------------------------------
-------------------------------------------------------------------
--------------- cosine ---------------
Accuracy for cosine: 0.5227963525835866
F1 score for cosine: 0.686626746506986
-------------------------------------------------------------------
-------------------------------------------------------------------


In [36]:
from skimage.metrics import structural_similarity as ssim

def classify_image_ssim(test_image, clean_image, adversarial_image, threshold=0.9):
    # Calculate SSIM between test image and clean image
    ssim_clean = ssim(test_image, clean_image, data_range=test_image.max() - test_image.min())

    # Calculate SSIM between test image and adversarial image
    ssim_adv = ssim(test_image, adversarial_image, data_range=test_image.max() - test_image.min())

    print(f"SSIM to Clean: {ssim_clean}, SSIM to Adversarial: {ssim_adv}")

    # Compare SSIM values and classify
    if ssim_clean > ssim_adv:
        return "Clean"
    else:
        return "Adversarial"
    
def classify_image_sum(test_attn, mean_attns_cln, mean_attns_adv): #np.sum((test_attn - mean_attns_cln)) #FIXME:
    # Flatten the images to 1D arrays (if not already flattened)
    test_attn_flat = test_attn.flatten()
    mean_attns_cln_flat = mean_attns_cln.flatten()
    mean_attns_adv_flat = mean_attns_adv.flatten()
    # Calculate cosine similarities
    distance_to_normal = np.sum((test_attn_flat - mean_attns_cln_flat))
    distance_to_adversarial = np.sum((test_attn_flat - mean_attns_adv_flat))

    print(f"distance_to_normal: {distance_to_normal}", f"distance_to_adversarial: {distance_to_adversarial}")
    # Compare distances and classify
    if distance_to_normal < distance_to_adversarial:
        return "Clean"
    else:
        return "Adversarial"
    
from scipy.spatial.distance import euclidean
# Defining function
def classify_image_euclidean(test_attn, mean_attns_cln, mean_attns_adv): # np.sqrt(np.sum(np.square(test_attn - mean_attns_cln)))
    # Flatten the images to 1D arrays (if not already flattened)
    test_attn_flat = test_attn.flatten()
    mean_attns_cln_flat = mean_attns_cln.flatten()
    mean_attns_adv_flat = mean_attns_adv.flatten()
    # Calculate Euclidean distances
    distance_to_normal = euclidean(test_attn_flat, mean_attns_cln_flat)
    distance_to_adversarial = euclidean(test_attn_flat, mean_attns_adv_flat)
    
    print(f"distance_to_normal: {distance_to_normal}", f"distance_to_adversarial: {distance_to_adversarial}")
    # Compare distances and classify
    if distance_to_normal < distance_to_adversarial:
        return "Clean"
    else:
        return "Adversarial"
    
from sklearn.metrics.pairwise import cosine_similarity
# Defining function
def classify_image_cos_similarity(test_attn, mean_attns_cln, mean_attns_adv): #np.dot(test_image,normal_image)/(norm(test_image)*norm(normal_image))
    # Flatten the images to 1D arrays (if not already flattened)
    test_attn_flat = test_attn.flatten()
    mean_attns_cln_flat = mean_attns_cln.flatten()
    mean_attns_adv_flat = mean_attns_adv.flatten()
    # Calculate cosine similarities
    similarity_to_normal = cosine_similarity([test_attn_flat], [mean_attns_cln_flat])
    similarity_to_adversarial = cosine_similarity([test_attn_flat], [mean_attns_adv_flat])

    print(f"similarity_to_normal: {similarity_to_normal}", f"similarity_to_adversarial: {similarity_to_adversarial}")
    # Compare similarity scores and classify
    if similarity_to_normal > similarity_to_adversarial:
        return "Clean"
    else:
        return "Adversarial"

In [45]:
num_test_images = len(test_attns['clean'])
test_attns_ = test_attns['clean']
mean_attns_cln = mean_attns['clean']
mean_attns_adv = mean_attns['PGD']

classifications = []
# Test each test image
for test_attn in test_attns_:
    result = classify_image_euclidean(test_attn, mean_attns_cln, mean_attns_adv)
    classifications.append(result)

# Calculate accuracy
true_labels = ["Clean"] * num_test_images  # Assuming all test images should be classified as "Normal"
accuracy = sum(1 for true, predicted in zip(true_labels, classifications) if true == predicted) / num_test_images
print(f"Overall Accuracy: {accuracy * 100:.2f}%")


distance_to_normal: 0.192083477973938 distance_to_adversarial: 0.20396122336387634
distance_to_normal: 0.371354877948761 distance_to_adversarial: 0.3879663944244385
distance_to_normal: 0.8472873568534851 distance_to_adversarial: 0.8225353956222534
distance_to_normal: 0.2801220715045929 distance_to_adversarial: 0.26920953392982483
distance_to_normal: 0.20168381929397583 distance_to_adversarial: 0.23497450351715088
distance_to_normal: 0.33897244930267334 distance_to_adversarial: 0.3112289011478424
distance_to_normal: 0.1640082448720932 distance_to_adversarial: 0.17700228095054626
distance_to_normal: 0.35717907547950745 distance_to_adversarial: 0.3632255792617798
distance_to_normal: 0.4455663859844208 distance_to_adversarial: 0.40794873237609863
distance_to_normal: 0.46297046542167664 distance_to_adversarial: 0.4984496533870697
distance_to_normal: 0.2762802839279175 distance_to_adversarial: 0.31832167506217957
distance_to_normal: 0.3410625755786896 distance_to_adversarial: 0.3929967880249